# Run Neural Net 

This script is implementation of original model of semantics cognition. 

The main part of the script is Net, which represents the main NN for cognition.

Output for the script is representation layers trained at given epoch, in form of numpy array.

In [16]:
# Import libraries
from __future__ import print_function
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
from torch.nn.functional import sigmoid, relu
from scipy.cluster.hierarchy import dendrogram, linkage
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [17]:
!ls "/content/drive/My Drive/1016 Cognition Project/"
path = "/content/drive/My Drive/1016 Cognition Project/"

 0429_1.csv			    rep_array_phase2.npy
 0429_2.csv			    rep_array_phase3.npy
 0509_1.csv			   'run_neural_net extension.ipynb'
 All_concepts_and_features.xlsx     run_neural_net.ipynb
 animal_only.csv		    sem_attributes_0429_1.txt
 concept_feature_sub.csv	    sem_attributes_0429_2.txt
 Concept_similarity_matrix.xlsx     sem_attributes_0509_1.txt
 data_cleaning_animals.ipynb	    sem_attributes_animal_only.txt
 data_cleaning.ipynb		    sem_attributes.txt
'draw graph 0511.ipynb'		    sem_data_0429_1.txt
 draw_graph.ipynb		    sem_data_0429_2.txt
 encoded_animal_only.csv	    sem_data_0509_1.txt
 encoded_data.csv		    sem_data_animal_only.txt
 Extension_Association.ipynb	    sem_data.txt
 images				    sem_items_0429_1.txt
'Image vector.ipynb'		    sem_items_0429_2.txt
 rep_array_2.npy		    sem_items_0509_1.txt
 rep_array_3.npy		    sem_items_animal_only.txt
 rep_array_animal_association.npy   sem_items.txt
 rep_array_animal_orig.npy	    sem_relations_0429_1.txt
 rep_array_animal_weight10

In [18]:
with open(path+'sem_items_0429_1.txt','r') as fid:
    names_items = np.array([l.strip() for l in fid.readlines()])
with open(path+'sem_relations_0429_1.txt','r') as fid:
    names_relations = np.array([l.strip() for l in fid.readlines()])
with open(path+'sem_attributes_0429_1.txt','r') as fid:
    names_attributes = np.array([l.strip() for l in fid.readlines()])
        
nobj = len(names_items)
nrel = len(names_relations)
nattributes = len(names_attributes)
print('List of items:')
print(names_items)
print("List of relations:")
print(names_relations)
print("List of attributes:")
print(names_attributes)

List of items:
['A variety of Chinese cabbage' 'African elephant' 'Alpaca' 'Bacon' 'Belt'
 'Blueberry' 'Broccoli' 'Bungarus multicinctus' 'Bus' 'Cabbage' 'Carp'
 'Carriage' 'Cauliflower' 'Cheetah' 'Cherry' 'Chow Chow' 'Chrysanthemum'
 'Clock' 'Cock' 'Coconut' 'Crucian carp' 'Cucurbit' 'Cushion' 'Cypress'
 'Daisy' 'Dinosaur' 'Dionaea muscipula' 'Dog' 'Durian' 'Elk' 'Ewes'
 'Glass' 'Grape' 'Guppy' 'Hami melon' 'Hawthorn' 'Hen' 'Honey peach'
 'Honeysuckle' 'Jellyfish' 'Jujube' 'Knife' 'Koala' 'Leech' 'Lettuce'
 'Loquat' 'Lotus' 'Metasequoia' 'Monkey' 'Mulberry' 'Olive' 'Orange'
 'Otarriinae' 'Oysters' 'Peach' 'Persian cat' 'Python' 'Rabbit' 'Raccoons'
 'Rat' 'Refrigerator' 'Robins' 'Sausage' 'Scallion' 'Sea snake' 'Seagull'
 'Seaweed' 'Siberian Husky' 'Snail' 'Soybean' 'Soybean Milk' 'Squab'
 'Sunflower' 'Swan' 'The Dachshund' 'The sperm whale' 'Timber wolf'
 'Tomatoes' 'Totoro' 'Touring car' 'Turbot' 'Tyrannosaurus Rex' 'Viper'
 'Vulture' 'Watermelon' 'Yak' 'babysbreath' 'belt' 'birch' '

In [19]:
print("number of items: ", nobj)
print("number of attributes: ", nattributes)

number of items:  157
number of attributes:  527


In [20]:
D = np.loadtxt(path+'sem_data_0429_1.txt')
input_pats = D[:,:nobj+nrel]
input_pats = torch.tensor(input_pats,dtype=torch.float)
output_pats = D[:,nobj+nrel:]
output_pats = torch.tensor(output_pats,dtype=torch.float)
N = input_pats.shape[0] # number of training patterns
input_v = input_pats[0,:].numpy().astype('bool')
output_v = output_pats[0,:].numpy().astype('bool')
print('Example input pattern:')
print(input_v.astype('int'))
print('Example output pattern:')
print(output_v.astype('int'))
print("")
print("Which encodes...")
print('Item ',end='')
print(names_items[input_v[:nobj]])
print('Relation ',end='')
print(names_relations[input_v[nobj:]])
print('Attributes ',end='')
print(names_attributes[output_v])

Example input pattern:
[1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0]
Example output pattern:
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 

In [ ]:
for i in range(N):
    input_v = input_pats[i,:].numpy().astype('bool')
    output_v = output_pats[i,:].numpy().astype('bool')
    print(names_items[input_v[:nobj]],names_relations[input_v[nobj:]],names_attributes[output_v])


['A variety of Chinese cabbage'] ['can'] ['stir']
['A variety of Chinese cabbage'] ['has'] ['leaves' 'nutrition' 'roots']
['A variety of Chinese cabbage'] ['is'] ['cabbage' 'food' 'green' 'green vegetable' 'plant' 'vegetables' 'white']
['A variety of Chinese cabbage'] ['like'] ['air' 'clean' 'planting' 'soil' 'sunlight' 'water']
['African elephant'] ['can'] ['view']
['African elephant'] ['has'] ['big ears' 'ears' 'ivory' 'limbs' 'long nose' 'mouth' 'nose' 'tail']
['African elephant'] ['is'] ['african' 'animals' 'big' 'elephant' 'mammals']
['African elephant'] ['like'] ['air' 'food' 'protection' 'to sleep' 'water']
['Alpaca'] ['can'] ['run']
['Alpaca'] ['has'] ['ears' 'eyes' 'hair' 'limbs' 'mouth' 'neck' 'tail']
['Alpaca'] ['is'] ['animals' 'cute' 'food' 'mammals' 'white']
['Alpaca'] ['need'] ['camel' 'sheep']
['Alpaca'] ['like'] ['air' 'food' 'grass' 'to sleep' 'water']
['Bacon'] ['can'] ['grill' 'stir']
['Bacon'] ['has'] ['fat' 'oil' 'salt']
['Bacon'] ['is'] ['food' 'meat' 'pork*' 're

In [21]:
class Net(nn.Module):
    def __init__(self, rep_size, hidden_size):
        super(Net, self).__init__()
        # Input
        #  rep_size : number of hidden units in "Representation Layer"
        #  hidden_Size : number of hidden units in "Hidden Layer"
        #
        # TODO : YOUR CODE GOES HERE
        self.fc_items_to_representation = nn.Linear(nobj, rep_size)
        self.fc_representation_relation_to_hidden = nn.Linear(rep_size + nrel, hidden_size)
        self.fc_hidden_to_attribute = nn.Linear(hidden_size, nattributes)
        
        #raise Exception('Replace with your code.')

    def forward(self, x):
        # Defines forward pass for the network on input patterns x
        #
        # Input can take these two forms:
        #
        #   x: [nobj+nrel 1D Tensor], which is a single input pattern as a 1D tensor
        #      (containing both object and relation 1-hot identifier) (batch size is B=1)
        #   OR
        #   x : [B x (nobj+nrel) Tensor], which is a batch of B input patterns (one for each row)
        #
        # Output
        #   output [B x nattribute Tensor], which is the output pattern for each input pattern B on the Attribute Layer
        #   hidden [B x hidden_size Tensor], which are activations in the Hidden Layer
        #   rep [B x rep_size Tensor], which are the activations in the Representation LAyer
        x = x.view(-1,nobj+nrel) # reshape as size [B x (nobj+nrel) Tensor] if B=1
        x_item = x[:,:nobj] # input to Item Layer [B x nobj Tensor]
        x_rel = x[:,nobj:] # input to Relation Layer [B x nrel Tensor]
        # TODO : YOUR CODE GOES HERE
        # ----
        netinput_rep = self.fc_items_to_representation(x_item)
        rep = relu(netinput_rep)
        rep_rel = torch.cat((rep, x_rel),1) 
        netinput_hidden = self.fc_representation_relation_to_hidden(rep_rel)
        hidden = relu(netinput_hidden)
        netinput_attribute = self.fc_hidden_to_attribute(hidden)
        output = sigmoid(netinput_attribute)
        
        #raise Exception('Replace with your code.')
        # -----
        return output, hidden, rep

In [24]:
def train(mynet,epoch_count,nepochs_additional=5000):
    # Input
    #  mynet : Net class object
    #  epoch_count : (scalar) how many epochs have been completed so far
    #  nepochs_additional : (scalar) how many more epochs we want to run
    mynet.train()
    for e in range(nepochs_additional): # for each epoch
        error_epoch = 0.
        perm = np.random.permutation(N)
        for p in perm: # iterate through input patterns in random order
            mynet.zero_grad() # reset gradient
            output, hidden, rep = mynet(input_pats[p,:]) # forward pass
            target = output_pats[p,:] 
            #print output and compare with target
            #if e % 50 == 0:
                #print("target: ",names_attributes[target.numpy().astype('bool')])
                #print("output: ",names_attributes[np.where(output.detach().numpy()>0.5,1,0).astype('bool').flatten()])
            loss = criterion(output, target) # compute loss
            loss.backward() # compute gradient 
            optimizer.step() # update network parameters
            error_epoch += loss.item()
        error_epoch = error_epoch / float(N)        
        if e % 500 == 0:
            print('epoch ' + str(epoch_count+e) + ' loss ' + str(round(error_epoch,3)))
    return epoch_count + nepochs_additional

In [25]:
def get_rep(net):
    # Extract the hidden activations on the Representation Layer for each item
    # 
    # Input
    #  net : Net class object
    #
    # Output
    #  rep : [nitem x rep_size numpy array], where each row is an item
    input_clean = torch.zeros(nobj,nobj+nrel)
    for idx,name in enumerate(names_items):
        input_clean[idx,idx] = 1. # 1-hot encoding of each object (while Relation Layer doesn't matter)
    output, hidden, rep = mynet(input_clean)
    return rep.detach().numpy()

def plot_rep(rep1,rep2,rep3,names):
    #  Compares Representation Layer activations of Items at three different times points in learning (rep1, rep2, rep3)
    #  using bar graphs
    # 
    #  Each rep1, rep2, rep3 is a [nitem x rep_size numpy array]
    #  names : [nitem list] of item names
    #
    nepochs_list = [nepochs_phase1,nepochs_phase2,nepochs_phase3]
    nrows = nobj
    R = np.dstack((rep1,rep2,rep3))    
    mx = R.max()
    mn = R.min()
    depth = R.shape[2]
    count = 1
    plt.figure(1,figsize=(4.2,360))
    for i in range(nrows):
        for d in range(R.shape[2]):
            plt.subplot(nrows, depth, count)
            rep = R[i,:,d]
            plt.bar(range(rep.size),rep)
            plt.ylim([mn,mx])
            plt.xticks([])
            plt.yticks([])        
            if d==0:
                plt.ylabel(names[i])
            if i==0:
                plt.title("epoch " + str(nepochs_list[d]))
            count += 1
    plt.show()

def plot_dendo(rep1,rep2,rep3,names):
    #  Compares Representation Layer activations of Items at three different times points in learning (rep1, rep2, rep3)
    #  using hierarchical clustering
    # 
    #  Each rep1, rep2, rep3 is a [nitem x rep_size numpy array]
    #  names : [nitem list] of item names
    #
    nepochs_list = [nepochs_phase1,nepochs_phase2,nepochs_phase3]
    linked1 = linkage(rep1,'single')
    linked2 = linkage(rep2,'single')
    linked3 = linkage(rep3,'single')
    mx = np.dstack((linked1[:,2],linked2[:,2],linked3[:,2])).max()+0.1    
    plt.figure(2,figsize=(24, 20))
    plt.subplot(3,1,1)    
    dendrogram(linked1, labels=names, color_threshold=0)
    plt.ylim([0,mx])
    plt.title('Hierarchical clustering; ' + "epoch " + str(nepochs_list[0]))
    plt.ylabel('Euclidean distance')
    plt.subplot(3,1,2)
    plt.title("epoch " + str(nepochs_list[1]))
    dendrogram(linked2, labels=names, color_threshold=0)
    plt.ylim([0,mx])
    plt.subplot(3,1,3)
    plt.title("epoch " + str(nepochs_list[2]))
    dendrogram(linked3, labels=names, color_threshold=0)
    plt.ylim([0,mx])
    plt.show()

In [26]:
learning_rate = 0.1
criterion = nn.MSELoss() # mean squared error loss function
mynet = Net(rep_size=8,hidden_size=15)
optimizer = torch.optim.SGD(mynet.parameters(), lr=learning_rate) # stochastic gradient descent

nepochs_phase1 = 500
nepochs_phase2 = 1000
nepochs_phase3 = 2500
nepochs_phase4 = 5000
epoch_count = 0
epoch_count = train(mynet,epoch_count,nepochs_additional=nepochs_phase1)
rep1 = get_rep(mynet)
epoch_count = train(mynet,epoch_count,nepochs_additional=nepochs_phase2-nepochs_phase1)
rep2 = get_rep(mynet)
epoch_count = train(mynet,epoch_count,nepochs_additional=nepochs_phase3-nepochs_phase2)
rep3 = get_rep(mynet)
epoch_count = train(mynet,epoch_count,nepochs_additional=nepochs_phase4-nepochs_phase3)
rep4 = get_rep(mynet)

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1709: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:528: UserWarning: Using a target size (torch.Size([527])) that is different to the input size (torch.Size([1, 527])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


epoch 0 loss 0.247
epoch 500 loss 0.007
epoch 1000 loss 0.007
epoch 1500 loss 0.007
epoch 2000 loss 0.006
epoch 2500 loss 0.006
epoch 3000 loss 0.005
epoch 3500 loss 0.005
epoch 4000 loss 0.005
epoch 4500 loss 0.005


In [ ]:
#plot_rep(rep1,rep2,rep3,names_items)
#plot_dendo(rep1,rep2,rep3,names_items)

In [27]:
rep4.shape

(157, 8)

In [28]:
np.save(path+"rep_array_phase1.npy", rep1)
np.save(path+"rep_array_phase2.npy", rep2)
np.save(path+"rep_array_phase3.npy", rep3)
np.save(path+"rep_array_phase4.npy", rep4)